In [1]:
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions
import requests
import sqlite3
import pandas as pd
import time
import csv
import json

In [2]:
# MS edge
options=EdgeOptions()
options.use_chromium=True
driver= Edge(options=options)

# Chrome or firefox
# from selenium import webdriver
# driver = webdriver.Chrome()


In [3]:
base_url='https://www.ajio.com'
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.55'}
driver.get(base_url)

In [4]:
conn=sqlite3.connect('GroomProducts.db')
c=conn.cursor()
# c.execute('''CREATE TABLE ProductList(Website TEXT, ProductLink TEXT,ProductName TEXT, 
#            ProductBrand TEXT, ProductCat TEXT, Sizes,Price INTEGER,MRP INTEGER, Gender TEXT,Description TEXT, PrimImgLink TEXT, SecImgLinks)''')



###  Get Products links list of a given search term

In [5]:
def get_prod_links(search_term):
    productsLink=[]
    search_term=search_term.replace(' ','%20')
    url='https://www.ajio.com/search/?text={}'.format(search_term)
    driver.get(url)
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;") 
    i = 1
    while True:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        soup=BeautifulSoup(driver.page_source,'html.parser')
        prod_List=soup.find_all('div','item rilrtl-products-list__item item')
        for item in prod_List:
            for link in item.find_all('a','rilrtl-products-list__link'):
                if (base_url+link['href']) not in productsLink:
                    productsLink.append(base_url+link['href'])
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break

    return productsLink

## Storing in Sqlite3 database

In [7]:
# search_prod_list=['men shirt' , 'men jeans' , 'men trouser', 'men underwear','men Vests' ,'men jacket','women sarees','women kaften', 'women top' , 'women leggings','women short', 'women dress', 'women jacket',
#                   'women innerwear','men shoes','women footwear', 'men watches','women watches','men wallets','women handbags','men backpacks','women backpacks','men ties','pocket squares',
#                   'men caps','women caps','men Mufflers','women Mufflers','women Gloves and masks','Mobile cases','women rings','women Jewellery','men socks','women socks']

# search_prod_list=['men shoes','women caps','Mobile cases']  # already executed on trial bases, 4000 items stored successfully. 
search_prod_list =['men jacket']   #this is demo item list , u can uncomment the above list and can run program on this list ...
                                              # but uncommenting that is not recommended, coz in beetween u lost the connection from server
                                              # and this lead to generate error in program, so it is safer that u run program at a time only on 2-3 products list

In [8]:
for item in search_prod_list:
    productsLinkList=get_prod_links(item)
    print(item,len(productsLinkList))
    category=item                                                                 # category --- 5

    search_term=item.replace(' ','%20')
       
    for link in productsLinkList:
        website=str('https://www.ajio.com/search/?text={}'.format(search_term))    #website....1
        product_link =link                                                         # product link ..2
        
        driver.get(link)
        soup=BeautifulSoup(driver.page_source,'html.parser')

        try:
            name=soup.find('h1',class_='prod-name').text.strip()                    #Product name...3
        except:
            name=category
        
        try:
            brand=soup.find('h2',class_='brand-name').text.strip()                   # brand...4

        except:
            brand='N.A.'
        
        try:
            size=soup.find('div','size-variant-block')                               # size.........6
            sizes = size.findChildren('div', {'size-swatch'}, recursive=True)
            size_l =[]
            for item in sizes:
                size_l.append(item.text.strip())
            size_list=json.dumps(size_l)                                              #..... 6...size array

        except:
            size_list='NA'
    
        try:
            sprice=soup.find('div',class_='prod-sp').text.strip()                  # price   ...7
            price=int(sprice.replace('Rs. ','').replace(',',''))

        except:
            price=0
            
        try:
            mrp=soup.find('span',class_='prod-cp').text.strip()                   # MRP....8
            MRP=int(mrp.replace('Rs. ','').replace(',',''))
        except:
            MRP=0

        
        if 'women' in item:                                                      # 9...gender
            gender='Female'
        elif 'men' in item:
            gender='Male'
    
        elif item=='Mobile cases':
            gender='For both'
        else:
            gender='Male'                                                       # 9...gender
        

        
        try:
            desc=soup.find('ul',class_='prod-list')                              # 10....desc
            descr=[]
            for item in desc:
                descr.append(item.text.strip())

            description=", ".join(descr)                                          
             
        except:
            description='Not available'                                         # 10...desc 
        
        try:
            images = soup.find('div', 'slick-track')
            all_images = images.findChildren("img" , recursive=True)

            secondary_images_links = []
            prim_image=all_images[0]['src']
    
            for  image in all_images[1:]:
                secondary_images_links.append(image['src'])

            sec_img=json.dumps(list(set(secondary_images_links)))

        except:
            prim_image='Not available'
            sec_img='Not available'
        
#         result=(website,product_link,name,brand,category,size_list,price,MRP,gender,description,prim_image,sec_img)
#         records.append(result)
        print(f"saving------{count}---")
        count+=1
        c.execute('''INSERT INTO ProductList values(?,?,?,?,?,?,?,?,?,?,?,?)''',
                 (website,product_link,name,brand,category,size_list,price,MRP,gender,description,prim_image,sec_img))
        conn.commit()


KeyboardInterrupt: 

In [ ]:
conn.close()             #close the connection with database

## Sqlite database viewer    https://inloop.github.io/sqlite-viewer/

## Storing data in CSV file

In [17]:
'''count=1
products=[]
search_prod_list=['men watches']



for item in search_prod_list:
    records=[]
    search_term=item.replace(' ','%20')
    category=item
    productsLinkList=get_prod_links(item)
    print(item,len(productsLinkList))
    
    for link in productsLinkList:
        

        website=str('https://www.ajio.com/search/?text={}'.format(search_term))             #website...1

        product_link =link                                                                # product link ....2
        
        driver.get(link)
        soup=BeautifulSoup(driver.page_source,'html.parser')

        try:
            name=soup.find('h1',class_='prod-name').text.strip()   #Product name...3
#         print(name)
        except:
            name=category
        try:
            brand=soup.find('h2',class_='brand-name').text.strip()    # brand...4
#         print(brand)
#         category=item   #   ....men clothing, women clothing, footwear..etc if no way to find, .item name hi likh denge....5
#         print(category)
        except:
            brand=''
        try:
            size=soup.find('div','size-variant-block')                      # size............6
            sizes = size.findChildren('div', {'size-swatch'}, recursive=True)
            size_l =[]
            for item in sizes:
                size_l.append(item.text.strip())
            size_list=json.dumps(size_l)                          #..... 6...size array
#         print(size_list)
        except:
            size_list='All size'
    
        try:
            sprice=soup.find('div',class_='prod-sp').text.strip()  # 7...price=int(sprice).Rs. 209 andRs. 2,499 inko int bnana hai....7..
            price=int(sprice.replace('Rs. ','').replace(',',''))
#         print(price)
        except:
            price='0'
        try:
            mrp=soup.find('span',class_='prod-cp').text.strip()    # MRP....8...integer bnana hai  Rs. 2,499
            MRP=int(mrp.replace('Rs. ','').replace(',',''))
        except:
            mrp='0'                                                
            MRP=int(mrp.replace('Rs. ','').replace(',',''))
#         print(MRP)
        # 9...gender....search item name mai if item.find('male'): gender male, elseif female, else both or in some cases again female
        
        gender='male'
#         print(gender)
        
        try:
            desc=soup.find('ul',class_='prod-list')   # 10....desc
            descr=[]
            for item in desc:
                descr.append(item.text.strip())

            description=",".join(descr)             # 10...desc ..string ...done
#         print(description)
#         prim_img=soup.find('img','rilrtl-lazy-img img-alignment zoom-cursor rilrtl-lazy-img-loaded').text
#         prim_image=prim_img['src']              # 11 ...primary image...done
        
        except:
            description=''
          # seconday image.....list hai, with help json.dumps karke array bna ke insert karna  ....12 ....
        try:
            images = soup.find('div', 'slick-track')
            all_images = images.findChildren("img" , recursive=True)

            secondary_images_links = []
            prim_image=all_images[0]['src']
    #         print(prim_image)
            for  image in all_images[1:]:
                secondary_images_links.append(image['src'])
    #         for links in secondary_images_links:
    #             print(links) 
            sec_img=json.dumps(secondary_images_links)
#         print(sec_img)
        except:
            prim_image=''
            sec_img=''
        
        result=(website,product_link,name,brand,category,size_list,price,MRP,gender,description,prim_image,sec_img)
        records.append(result)

        print(f"saving------{count}---")
        count+=1
        with open('results.csv','w',newline='',encoding='utf8') as f:
                writer=csv.writer(f)
                writer.writerow(['Website','Prod_Url','Name','Brand','Prod_cat','Sizes','Price','MRP','Gender','Description','PrimaryImg','Sec_imag'])
                writer.writerows(records)        
'''

'count=1\nproducts=[]\nsearch_prod_list=[\'men watches\']\n\n\n\nfor item in search_prod_list:\n    records=[]\n    search_term=item.replace(\' \',\'%20\')\n    category=item\n    productsLinkList=get_prod_links(item)\n    print(item,len(productsLinkList))\n    \n    for link in productsLinkList:\n        \n\n        website=str(\'https://www.ajio.com/search/?text={}\'.format(search_term))             #website...1\n\n        product_link =link                                                                # product link ....2\n        \n        driver.get(link)\n        soup=BeautifulSoup(driver.page_source,\'html.parser\')\n\n        try:\n            name=soup.find(\'h1\',class_=\'prod-name\').text.strip()   #Product name...3\n#         print(name)\n        except:\n            name=category\n        try:\n            brand=soup.find(\'h2\',class_=\'brand-name\').text.strip()    # brand...4\n#         print(brand)\n#         category=item   #   ....men clothing, women clothing, footw

# reading data from database using pandas or u can drag and drop database here:- https://inloop.github.io/sqlite-viewer/

In [18]:
# df=pd.read_sql_query("SELECT * FROM TryProductList ",conn)
# print(df.head(20))

# conn.close()